In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [2]:
with open('trainingset.csv', 'r') as csvfile:
    df = pd.read_csv(csvfile, dtype={'booking_bool':np.int32,'srch_destination_id':np.int32, 
                                     'site_id':np.int32, 'visitor_location_country_id':np.int32,
                                     'visitor_hist_starrating':np.float, 'visitor_hist_adr_usd':np.float,
                                     'prop_country_id':np.int32, 'prop_id':np.int32, 
                                     'prop_starrating':np.int32, 'prop_review_score':np.float,
                                     'prop_brand_bool':np.int32, 'prop_location_score1':np.float,
                                     'prop_location_score2':np.float, 'prop_log_historical_price':np.float,
                                     'price_usd':np.float, 'promotion_flag':np.int32,\
                                     'srch_length_of_stay':np.int32, 'srch_booking_window':np.int32,
                                     'srch_adults_count':np.int32, 'srch_children_count':np.int32,
                                     'srch_room_count':np.int32, 'srch_saturday_night_bool':np.int32,
                                     'srch_query_affinity_score':np.float, 'orig_destination_distance':np.float,
                                     'random_bool':np.int32, 'position':np.int32, 'click_bool':np.int32,
                                     'booking_bool':np.int32, 'gross_bookings_usd':np.float})

In [3]:
with open('testset.csv', 'r') as csvfile:
    dfTest = pd.read_csv(csvfile, dtype={'booking_bool':np.int32,'srch_destination_id':np.int32, 
                                     'site_id':np.int32, 'visitor_location_country_id':np.int32,
                                     'visitor_hist_starrating':np.float, 'visitor_hist_adr_usd':np.float,
                                     'prop_country_id':np.int32, 'prop_id':np.int32, 
                                     'prop_starrating':np.int32, 'prop_review_score':np.float,
                                     'prop_brand_bool':np.int32, 'prop_location_score1':np.float,
                                     'prop_location_score2':np.float, 'prop_log_historical_price':np.float,
                                     'price_usd':np.float, 'promotion_flag':np.int32,\
                                     'srch_length_of_stay':np.int32, 'srch_booking_window':np.int32,
                                     'srch_adults_count':np.int32, 'srch_children_count':np.int32,
                                     'srch_room_count':np.int32, 'srch_saturday_night_bool':np.int32,
                                     'srch_query_affinity_score':np.float, 'orig_destination_distance':np.float,
                                     'random_bool':np.int32})

In [12]:
df = pd.read_csv('testset.csv')

### Dates

In [4]:
df["date_time"] = pd.to_datetime(df["date_time"])
df["year"] = df["date_time"].dt.year
df["month"] = df["date_time"].dt.month

dfTest["date_time"] = pd.to_datetime(dfTest["date_time"])
dfTest["year"] = dfTest["date_time"].dt.year
dfTest["month"] = dfTest["date_time"].dt.month

### Remove outliers price_usd

In [4]:
# First without due to loss of values, maybe later with

# top2point5 = df.price_usd.quantile(0.975)
# low2point5 = df.price_usd.quantile(0.025)
# df = df[df.price_usd < top2point5]
# df = df[df.price_usd > low2point5]

# top2point5test = dfTest.price_usd.quantile(0.975)
# low2point5test = dfTest.price_usd.quantile(0.025)
# dfTest = dfTest[df.price_usd < top2point5]
# dfTest = dfTest[df.price_usd > low2point5]

### Fill in missing values with worst case scenario

In [7]:
df.prop_location_score2.loc[np.isnan(df.prop_location_score2)] = 0

dfTest.prop_location_score2.loc[np.isnan(dfTest.prop_location_score2)] = 0

In [8]:
df.visitor_hist_starrating.loc[np.isnan(df.visitor_hist_starrating)] = 0
df.visitor_hist_adr_usd.loc[np.isnan(df.visitor_hist_adr_usd)] = 0

dfTest.visitor_hist_starrating.loc[np.isnan(dfTest.visitor_hist_starrating)] = 0
dfTest.visitor_hist_adr_usd.loc[np.isnan(dfTest.visitor_hist_adr_usd)] = 0

In [9]:
df.orig_destination_distance[np.isnan(df.orig_destination_distance)] = 0 
df.prop_review_score[np.isnan(df.prop_review_score)] = 0
df.srch_query_affinity_score[np.isnan(df.srch_query_affinity_score)] = 0

dfTest.orig_destination_distance[np.isnan(dfTest.orig_destination_distance)] = 0 
dfTest.prop_review_score[np.isnan(dfTest.prop_review_score)] = 0
dfTest.srch_query_affinity_score[np.isnan(dfTest.srch_query_affinity_score)] = 0

In [10]:
df.iloc[:,27:51] = df.iloc[:,27:51].fillna(value = 0, axis = 0)

dfTest.iloc[:,27:51] = dfTest.iloc[:,27:51].fillna(value = 0, axis = 0)

### Composite features

In [11]:
df['prop_id_counts'] = df.groupby(['prop_id'])['prop_id'].transform('count')

dfTest['prop_id_counts'] = dfTest.groupby(['prop_id'])['prop_id'].transform('count')

In [12]:
df['usd_diff'] = abs(df.price_usd - df.visitor_hist_adr_usd)

dfTest['usd_diff'] = abs(dfTest.price_usd - dfTest.visitor_hist_adr_usd)

In [13]:
df['star_diff'] = abs(df.prop_starrating - df.visitor_hist_starrating) 

dfTest['star_diff'] = abs(dfTest.prop_starrating - dfTest.visitor_hist_starrating) 

In [14]:
df.prop_log_historical_price.loc[np.isnan(df.prop_log_historical_price)] = 0
df['diff_hist_price'] = np.exp(df.prop_log_historical_price) - df.price_usd

dfTest.prop_log_historical_price.loc[np.isnan(dfTest.prop_log_historical_price)] = 0
dfTest['diff_hist_price'] = np.exp(dfTest.prop_log_historical_price) - dfTest.price_usd

In [15]:
#Total price/per night
df['total_price'] = df.price_usd * df.srch_room_count

dfTest['total_price'] = dfTest.price_usd * dfTest.srch_room_count

In [16]:
#Price per person
df['price_per_pers'] = (df.price_usd * df.srch_room_count)/(df.srch_adults_count + df.srch_children_count)

dfTest['price_per_pers'] = (dfTest.price_usd * dfTest.srch_room_count)/(dfTest.srch_adults_count + dfTest.srch_children_count)

In [17]:
df['prop_score'] = (df.prop_location_score2 + 0.00001)/(df.prop_location_score1 + 0.00001)

dfTest['prop_score'] = (dfTest.prop_location_score2 + 0.00001)/(dfTest.prop_location_score1 + 0.00001)

In [18]:
df['nr_pers'] =  df.srch_adults_count + df.srch_children_count

dfTest['nr_pers'] =  dfTest.srch_adults_count + dfTest.srch_children_count

### Rank features

In [19]:
df['price_rank'] = df.groupby('srch_id', sort=False)['price_usd'].rank(ascending=True, method ="min")
df['stars_rank'] = df.groupby('srch_id', sort=False)['prop_starrating'].rank(ascending=True, method = "min")
df['score_rank'] = df.groupby('srch_id', sort=False)['prop_location_score2'].rank(ascending=True, method = "min")

dfTest['price_rank'] = dfTest.groupby('srch_id', sort=False)['price_usd'].rank(ascending=True, method ="min")
dfTest['stars_rank'] = dfTest.groupby('srch_id', sort=False)['prop_starrating'].rank(ascending=True, method = "min")
dfTest['score_rank'] = dfTest.groupby('srch_id', sort=False)['prop_location_score2'].rank(ascending=True, method = "min")

### Estimate position:

In [18]:
print len(set(df.prop_id.unique()).intersection(set(dfTest.prop_id.unique())))
print len(df.prop_id.unique()), len(dfTest.prop_id.unique())
print len(set(dfTest.prop_id.unique()) - set(df.prop_id.unique()))


SyntaxError: invalid syntax (<ipython-input-18-ff7980ff934b>, line 1)

In [19]:
# df['position_estimate'] = df.groupby(['prop_id','month','srch_destination_id'])['position'].transform(np.mean)
# df.position_estimate = np.int32(df.position_estimate)

# If working with NcDG this might influence the scores of properties on certain search parameters which may include noise

In [21]:
#print df[['position','position_estimate']].head(100)
from sklearn.metrics import mean_absolute_error
print mean_absolute_error(df.position,df.position_estimate)

SyntaxError: invalid syntax (<ipython-input-21-7cf4cceb0f77>, line 3)

In [22]:
header = ['srch_id','prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'star_diff', 'srch_query_affinity_score', 'orig_destination_distance',
                'prop_review_score','position','price_rank','stars_rank','score_rank', 'booking_bool', 'click_bool', 'price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'comp2_rate', 'comp5_rate','comp8_rate']

In [23]:
df.to_csv('new_train.csv', columns = header)

In [29]:
df.position_estimate.isnull().any()

AttributeError: 'DataFrame' object has no attribute 'position_estimate'

In [23]:
header = ['srch_id','prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'prop_id', 'star_diff', 'srch_query_affinity_score', 'orig_destination_distance', 'prop_brand_bool',
                'prop_review_score','position','price_rank','stars_rank','score_rank', 'booking_bool', 'click_bool', 'price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'comp2_rate', 'comp5_rate','comp8_rate']
header2 = ['srch_id','prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'prop_id', 'star_diff', 'srch_query_affinity_score', 'orig_destination_distance', 'prop_brand_bool',
                'prop_review_score','price_rank','stars_rank','score_rank','price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'comp2_rate', 'comp5_rate','comp8_rate']

In [6]:
# df.to_csv('train3.csv', columns = header)
# dfTest.to_csv('test3.csv', columns = header2)
df.to_csv('target3.csv', columns = ['position', 'rank_rel'] )

In [7]:
df.shape

(4958347, 55)

In [20]:
print df.columns
print dfTest.columns

Index([u'srch_id', u'date_time', u'site_id', u'visitor_location_country_id',
       u'visitor_hist_starrating', u'visitor_hist_adr_usd', u'prop_country_id',
       u'prop_id', u'prop_starrating', u'prop_review_score',
       u'prop_brand_bool', u'prop_location_score1', u'prop_location_score2',
       u'prop_log_historical_price', u'position', u'price_usd',
       u'promotion_flag', u'srch_destination_id', u'srch_length_of_stay',
       u'srch_booking_window', u'srch_adults_count', u'srch_children_count',
       u'srch_room_count', u'srch_saturday_night_bool',
       u'srch_query_affinity_score', u'orig_destination_distance',
       u'random_bool', u'comp1_rate', u'comp1_inv', u'comp1_rate_percent_diff',
       u'comp2_rate', u'comp2_inv', u'comp2_rate_percent_diff', u'comp3_rate',
       u'comp3_inv', u'comp3_rate_percent_diff', u'comp4_rate', u'comp4_inv',
       u'comp4_rate_percent_diff', u'comp5_rate', u'comp5_inv',
       u'comp5_rate_percent_diff', u'comp6_rate', u'comp6_inv',
  

In [ ]:
df = df.drop(['date_time', axis=1)
dfTest = dfTest.drop('date_time', axis = 1)

['srch_id','prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'star_diff', 'srch_query_affinity_score', 'orig_destination_distance',
                'prop_review_score','position','price_rank','stars_rank','score_rank', 'booking_bool', 'click_bool', 'price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'comp2_rate', 'comp5_rate','comp8_rate']

### NDCG formula:

In [4]:
df['rank_rel'] = np.zeros(len(df))
df['rank_rel'] = np.where(df.click_bool == 1, 1, df.rank_rel)
df['rank_rel'] = np.where(df.booking_bool == 1, 5, df.rank_rel)

# df['ndcgscore'] = ((2**df.ndcg - 1) / (np.log2(df.position + 1)))

In [25]:
df.ndcg

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         5.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
          ... 
4958317    0.0
4958318    0.0
4958319    0.0
4958320    0.0
4958321    5.0
4958322    0.0
4958323    0.0
4958324    0.0
4958325    0.0
4958326    0.0
4958327    0.0
4958328    0.0
4958329    0.0
4958330    0.0
4958331    0.0
4958332    0.0
4958333    0.0
4958334    0.0
4958335    0.0
4958336    0.0
4958337    0.0
4958338    0.0
4958339    0.0
4958340    0.0
4958341    0.0
4958342    0.0
4958343    0.0
4958344    0.0
4958345    5.0
4958346    0.0
Name: ndcg, Length: 4958347, dtype: float64

In [36]:
df.to_csv('test2.csv')

In [41]:
df.groupby(['srch_id', 'position'])['ndcg', 'ndcgscore']

,ndcg,ndcgscore
0,0.0,0.000000
1,0.0,0.000000
2,0.0,0.000000
3,0.0,0.000000
4,0.0,0.000000
5,0.0,0.000000
6,0.0,0.000000
7,0.0,0.000000
8,0.0,0.000000
9,0.0,0.000000


Per search_id ncdg berekenen

Steps:
column with count of srch_id items


In [ ]:
df.groupby('srch_id').a

In [ ]:
for i in df.srch_id.unique():
    ncdg(srch_id)

def ncdg(srch_id):
    number_of_instances = df.srch_id[srch_id].count()
    
    
    
    
    
    
    return 

In [ ]:
df.srch

In [32]:
df2 = pd.read_csv('train2.csv')
df2.columns

Index(['Unnamed: 0', 'srch_id', 'prop_country_id', 'year', 'month',
       'prop_score', 'prop_location_score1', 'prop_location_score2',
       'diff_hist_price', 'usd_diff', 'star_diff', 'srch_query_affinity_score',
       'orig_destination_distance', 'prop_review_score', 'position',
       'price_rank', 'stars_rank', 'score_rank', 'booking_bool', 'click_bool',
       'price_per_pers', 'random_bool', 'price_usd', 'total_price', 'nr_pers',
       'prop_id_counts', 'comp2_rate', 'comp5_rate', 'comp8_rate', 'ndcg'],
      dtype='object')

In [35]:
df.columns

Index(['srch_id', 'prop_country_id', 'year', 'month', 'prop_score',
       'prop_location_score1', 'prop_location_score2', 'diff_hist_price',
       'usd_diff', 'star_diff', 'srch_query_affinity_score',
       'orig_destination_distance', 'prop_review_score', 'price_rank',
       'stars_rank', 'score_rank', 'price_per_pers', 'random_bool',
       'price_usd', 'total_price', 'nr_pers', 'prop_id_counts', 'comp2_rate',
       'comp5_rate', 'comp8_rate', 'ndcg'],
      dtype='object')

Computing NcDG

So position

In [ ]:
df['ncdg_rankingvalue'] = np.zeros(len(df))
df['ncdg_rankingvalue'] = np.where(df['click_bool'] == 1, 1, df['ncdg_rankingvalue'])
df['ncdg_rankingvalue'] = np.where(df['booking_bool'] == 1, 5, df['ncdg_rankingvalue'])

In [ ]:
dfsimpletest = df.sample(frac=0.1)

In [ ]:
dfsimpletarget = dfsimpletest[['position', 'ncdg_rankingvalue']]

In [ ]:
dfsimpletest = dfsimpletest.drop(['position', 'ncdg_rankingvalue'], axis =1)

In [ ]:
dfsimpletest = dfsimpletest.drop('date_time', axis =1)

### Rankpy

Need.
Trainingset, excluding position, click_bool, booking_bool, gross_bookings_usd, date_time

Validation set, containing single NCDG values derived from position, click_bool, booking_bool

Test set, same columns as trainingset with same transformations

All files need to be saved to txt format

In [ ]:
import pandas as pd
import numpy as np
import logging
from rankpy.queries import Queries
from rankpy.queries import find_constant_features
from rankpy.models import LambdaMART

In [ ]:
# Turn on logging.
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.INFO)

# Load the query datasets.
training_queries = Queries.load_from_text('data/MQ2007/Fold1/train.txt')
validation_queries = Queries.load_from_text('data/MQ2007/Fold1/vali.txt')
test_queries = Queries.load_from_text('data/MQ2007/Fold1/test.txt')

logging.info('================================================================================')

# Save them to binary format ...
training_queries.save('data/MQ2007/Fold1/training')
validation_queries.save('data/MQ2007/Fold1/validation')
test_queries.save('data/MQ2007/Fold1/test')

# ... because loading them will be then faster.
training_queries = Queries.load('data/MQ2007/Fold1/training')
validation_queries = Queries.load('data/MQ2007/Fold1/validation')
test_queries = Queries.load('data/MQ2007/Fold1/test')

logging.info('================================================================================')

# Print basic info about query datasets.
logging.info('Train queries: %s' % training_queries)
logging.info('Valid queries: %s' % validation_queries)
logging.info('Test queries: %s' %test_queries)

logging.info('================================================================================')

# Set this to True in order to remove queries containing all documents
# of the same relevance score -- these are useless for LambdaMART.
remove_useless_queries = False

# Find constant query-document features.
cfs = find_constant_features([training_queries, validation_queries, test_queries])

# Get rid of constant features and (possibly) remove useless queries.
training_queries.adjust(remove_features=cfs, purge=remove_useless_queries)
validation_queries.adjust(remove_features=cfs, purge=remove_useless_queries)
test_queries.adjust(remove_features=cfs)

# Print basic info about query datasets.
logging.info('Train queries: %s' % training_queries)
logging.info('Valid queries: %s' % validation_queries)
logging.info('Test queries: %s' % test_queries)

logging.info('================================================================================')

model = LambdaMART(metric='nDCG@10', max_leaf_nodes=7, shrinkage=0.1,
                   estopping=50, n_jobs=-1, min_samples_leaf=50,
                   random_state=42)

model.fit(training_queries, validation_queries=validation_queries)

logging.info('================================================================================')

logging.info('%s on the test queries: %.8f'
             % (model.metric, model.evaluate(test_queries, n_jobs=-1)))

model.save('LambdaMART_L7_S0.1_E50_' + model.metric)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('train2.csv')
df.columns

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'visitor_hist_starrating',
       'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score', 'orig_destination_distance', 'random_bool',
       'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate',
       'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff'

In [34]:
df.columns

Index(['srch_id', 'prop_country_id', 'year', 'month', 'prop_score',
       'prop_location_score1', 'prop_location_score2', 'diff_hist_price',
       'usd_diff', 'star_diff', 'srch_query_affinity_score',
       'orig_destination_distance', 'prop_review_score', 'price_rank',
       'stars_rank', 'score_rank', 'price_per_pers', 'random_bool',
       'price_usd', 'total_price', 'nr_pers', 'prop_id_counts', 'comp2_rate',
       'comp5_rate', 'comp8_rate', 'ndcg'],
      dtype='object')

In [10]:

df = df[['srch_id','prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
         'usd_diff', 'star_diff', 'srch_query_affinity_score', 'orig_destination_distance',
            'prop_review_score','position','price_rank','stars_rank','score_rank', 'booking_bool', 'click_bool', 'price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'comp2_rate', 'comp5_rate','comp8_rate', 'ndcg']]

In [8]:
df.ndcg.value_counts()

0.0    4736468
5.0     138390
1.0      83489
Name: ndcg, dtype: int64

In [33]:
df =df[['srch_id', 'prop_country_id', 'year', 'month',
       'prop_score', 'prop_location_score1', 'prop_location_score2',
       'diff_hist_price', 'usd_diff', 'star_diff', 'srch_query_affinity_score',
       'orig_destination_distance', 'prop_review_score',
       'price_rank', 'stars_rank', 'score_rank',
       'price_per_pers', 'random_bool', 'price_usd', 'total_price', 'nr_pers',
       'prop_id_counts', 'comp2_rate', 'comp5_rate', 'comp8_rate', 'ndcg']]